<hr>
<center><strong>  Taller de Optimización y Simulaciones <br> Simulación Montecarlo <br>Teoría de Colas<br>
<hr>
Taller 1
</strong></center>
<hr>
<center><strong> Hecho por Alejandro Uscátegui </strong></center>
<hr>
<center><strong> Universidad Javeriana <br> 2023 <br> Prof. Gabriel Zambrano  </strong></center>
<hr>

In [113]:
# Es necesario instalar pandas, numpy, openpyxl y scipy. Es posible hacer esto con pip.

import pandas as pd
import math
import numpy as np
from pulp import *
import xlsxwriter
import random
from openpyxl import load_workbook
import scipy.stats as st

<i>Indicamos las constantes, creamos la tabla e indicamos las variables</i>

1. Sabemos que se cumple la condición de estabilidad pues lambda es menor a miu


In [114]:
# Constantes

lambdaa = 0.05
miu = 0.08
acumuladordetiempos = 0

# Modelo de Little para el sistema de colas

TiempoPromedioEnSistemaTeorico = 1/(miu - lambdaa)
tiempoPromedioEnEspera = lambdaa/(miu*(miu - lambdaa))
tiempoPromedioServicio = 1/miu
utilizaciondelsistemaTeorico = lambdaa/miu
CantidadPromedioDeOrdenesEnSistemaTeorico = lambdaa/(miu - lambdaa)
CantidadPromedioOrdenesEnColaTeorico = (lambdaa*lambdaa)/(miu*(miu - lambdaa))

# Columna de número de replicas

Nreplicas = 1000

# Variables de la réplica

tiempos_espera_cola = []
tiempos_espera_sistema = []
promedios_cantidad_ordenes_sistema = []
promedios_cantidad_ordenes_cola = []


# Pensamos que debíamos hacer por listas el programa, pues así podríamos relacionar facilmente
# las cabeceras de cada columna, en vez de una matriz

for i in range(1000):

    clientes = [i+1 for i in range(200)]
    # print(clientes)

    llegada = [(-1/lambdaa)*math.log(random.random()) for i in range(200)]
    # print(llegada)

    relojLlegada = []

    for i in range(200):
        acumuladordetiempos += llegada[i]
        relojLlegada.append(acumuladordetiempos)
    # print(relojLlegada)

    tiempodeServicio = [(-1/miu)*math.log(random.random()) for i in range(200)]
    # print(tiempodeServicio)

    tiempoEnCola = []
    tiempoEnCola.append(0)

    inicioServicio = []
    inicioServicio.append(relojLlegada[0])

    findeServicio = []
    findeServicio.append(inicioServicio[0]+tiempodeServicio[0])

    for i in range(1, 200):
        # inicioServicio.append(relojLlegada) if relojLlegada[i]>(relojLlegada[i-1] + tiempodeServicio[i -1]) else inicioServicio.append(relojLlegada[i-1] + tiempodeServicio[i -1])

        if (relojLlegada[i] >= findeServicio[i-1]):
            tiempoEnCola.append(0)
        else:
            tiempoEnCola.append(findeServicio[i-1]-relojLlegada[i])

        if (tiempoEnCola[i] == 0):
            inicioServicio.append(relojLlegada[i])
        else:
            inicioServicio.append(findeServicio[i-1])

        findeServicio.append(inicioServicio[i] + tiempodeServicio[i])

    tiempoTotalSistema = [findeServicio[i] - relojLlegada[i]
                          for i in range(200)]

    tiempoReposoSistema = []
    tiempoReposoSistema.append(relojLlegada[0])

    for i in range(1, 200):
        tiempoReposoSistema.append(inicioServicio[i] - findeServicio[i-1])

    # Variables simuladas

    TiempoPromedioEnCola_Simulation = sum(tiempoEnCola)/len(tiempoEnCola)
    tiempoPromedioEnElSistema_Simulation = sum(
        tiempoTotalSistema)/len(tiempoTotalSistema)
    CantidadPromedioDeOrdenesEnSistema_Simulation = lambdaa * \
        tiempoPromedioEnElSistema_Simulation
    CantidadPromedioOrdenesEnCola_Simulation = lambdaa*TiempoPromedioEnCola_Simulation

    # Lista que recibe cada variable simulada

    
    tiempos_espera_cola.append(TiempoPromedioEnCola_Simulation)
    tiempos_espera_sistema.append(tiempoPromedioEnElSistema_Simulation)
    promedios_cantidad_ordenes_sistema.append(
        CantidadPromedioDeOrdenesEnSistema_Simulation)
    promedios_cantidad_ordenes_cola.append(CantidadPromedioOrdenesEnCola_Simulation)


# Línea para crear una matriz de prueba
matriz = np.column_stack((promedios_cantidad_ordenes_cola, promedios_cantidad_ordenes_sistema, tiempos_espera_cola, tiempos_espera_sistema))

In [115]:
writer=pd.ExcelWriter('replicas.xlsx', engine='xlsxwriter')
workbook=writer.book

dataf=pd.DataFrame([matriz[i,:] for i in range(1000)], columns=['promedios_cantidad_ordenes_cola', 'promedios_cantidad_ordenes_sistema', 'tiempos_espera_cola', 'tiempos_espera_sistema'])

dataf.to_excel(
    writer,
    index=False,
    sheet_name= 'prueba'
)


writer.save()

C:\Users\admin\AppData\Local\Temp\ipykernel_14128\3569160868.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


<i>El objetivo de esta última parte es generar en términos de código los intervalos de confianza </i>

In [116]:
intervalo_tiempo_espera_cola = st.t.interval(confidence= 0.95, df = len(tiempos_espera_cola)-1, loc =np.mean(tiempos_espera_cola), scale = st.sem(tiempos_espera_cola))
intervalo_tiempo_espera_sistema = st.t.interval(confidence= 0.95, df = len(tiempos_espera_sistema)-1, loc =np.mean(tiempos_espera_sistema), scale = st.sem(tiempos_espera_sistema))
intervalo_cantidad_ordenes_promedio_cola = st.t.interval(confidence= 0.95, df = len(promedios_cantidad_ordenes_cola)-1, loc =np.mean(promedios_cantidad_ordenes_cola), scale = st.sem(promedios_cantidad_ordenes_cola))
intervalo_cantidad_ordenes_promedio_sistema = st.t.interval(confidence= 0.95, df = len(promedios_cantidad_ordenes_sistema)-1, loc =np.mean(promedios_cantidad_ordenes_sistema), scale = st.sem(promedios_cantidad_ordenes_sistema))

print("Intervalo de confianza del 95% para la cantidad promedio de órdenes en cola es: ", intervalo_cantidad_ordenes_promedio_cola)
print("Intervalo de confianza del 95% para la cantidad promedio de órdenes en el sistema es: ", intervalo_cantidad_ordenes_promedio_sistema)
print("Intervalo de confianza del 95% para la cantidad promedio de tiempo que dura una orden en la cola es: ", intervalo_tiempo_espera_cola)
print("Intervalo de confianza del 95% para la cantidad promedio de tiempo que dura una orden en el sistema es: ",intervalo_tiempo_espera_sistema )

Intervalo de confianza del 95% para la cantidad promedio de órdenes en cola es:  (0.9888772578103492, 1.0454423147953078)
Intervalo de confianza del 95% para la cantidad promedio de órdenes en el sistema es:  (1.6151803038547539, 1.675290554915931)
Intervalo de confianza del 95% para la cantidad promedio de tiempo que dura una orden en la cola es:  (19.77754515620698, 20.908846295906155)
Intervalo de confianza del 95% para la cantidad promedio de tiempo que dura una orden en el sistema es:  (32.30360607709507, 33.50581109831862)


In [117]:


print("Según la ley de Little, la cantidad promedio de órdenes en cola es: ", CantidadPromedioOrdenesEnColaTeorico, 
      "\nla cantidad promedio de órdenes en el sistema es: ", CantidadPromedioDeOrdenesEnSistemaTeorico,
      "\nla cantidad promedio de tiempo que dura una orden en la cola es: ", tiempoPromedioEnEspera,
      "\nla cantidad promedio de tiempo que dura una orden en el sistema es: ", TiempoPromedioEnSistemaTeorico
)


Según la ley de Little, la cantidad promedio de órdenes en cola es:  1.041666666666667 
la cantidad promedio de órdenes en el sistema es:  1.6666666666666667 
la cantidad promedio de tiempo que dura una orden en la cola es:  20.833333333333336 
la cantidad promedio de tiempo que dura una orden en el sistema es:  33.333333333333336


In [118]:
""" # Exportamos resultados a excel (sólo para hacer pruebas temporales)

writer=pd.ExcelWriter('pruebas.xlsx', engine='xlsxwriter')
workbook=writer.book

dataf=pd.DataFrame([matriz[i,:] for i in range(200)], columns=['clientes', 'llegada', 'relojLlegada', 'tiempodeServicio', 'tiempoEnCola', 'inicioServicio', 'findeServicio', 'tiempoTotalSistema', 'tiempoReposoSistema'])

dataf.to_excel(
    writer,
    index=False,
    sheet_name= 'prueba'
)

writer.save() """

" # Exportamos resultados a excel (sólo para hacer pruebas temporales)\n\nwriter=pd.ExcelWriter('pruebas.xlsx', engine='xlsxwriter')\nworkbook=writer.book\n\ndataf=pd.DataFrame([matriz[i,:] for i in range(200)], columns=['clientes', 'llegada', 'relojLlegada', 'tiempodeServicio', 'tiempoEnCola', 'inicioServicio', 'findeServicio', 'tiempoTotalSistema', 'tiempoReposoSistema'])\n\ndataf.to_excel(\n    writer,\n    index=False,\n    sheet_name= 'prueba'\n)\n\nwriter.save() "